In [1]:
import pennylane as qml
import cv2
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import glob

In [2]:
import matplotlib.pyplot as plt

In [3]:
np.random.seed(42)

In [4]:
def import_image_to_array(image_path, image_name, image_type):
    if image_type == "gray":
        mode = 0
    else:
        mode = 1
    im = cv2.imread(image_path + "\\" + image_name, mode).astype('float64')
    return im

In [5]:
def import_multiple_images_to_array(directoryCats, directoryOther, filetype, image_type):
    images_labels = []
    images_array_cats = []
    images_array_other = []
    if image_type == "gray":
        mode = 0
    else:
        mode = 1
    filelistCats = glob.glob(directoryCats + "\\"+"*"+filetype)
    for myFile in filelistCats:
        image = cv2.imread(myFile,0)
        images_array_cats.append(image)
        images_labels.append(1)
    filelistOther = glob.glob(directoryOther + "\\"+"*"+filetype)
    for myFile in filelistOther:
        image = cv2.imread(myFile,0)
        images_array_other.append(image)
        images_labels.append(0)
    images_array = np.concatenate((images_array_cats, images_array_other), axis=0)
    return images_array, images_labels

In [6]:
image = import_image_to_array("C:\\Users\\dju10\\Pennylane", "cat.jpg", "gray")
image_normalized = image #image/255 if we want to actually notmalize the image values, not doing this for now

In [7]:
images, labels = import_multiple_images_to_array("C:\\Users\\dju10\\Pennylane\\Cats", "C:\\Users\\dju10\\Pennylane\\Other", ".jpg", "gray")
print(images)
print(len(images[0][0]))
print(labels)

[array([[255, 255, 255, ..., 139, 136, 134],
       [255, 255, 255, ..., 137, 134, 132],
       [255, 255, 255, ..., 136, 133, 131],
       ...,
       [244, 244, 244, ..., 243, 243, 243],
       [245, 244, 244, ..., 243, 243, 243],
       [245, 245, 245, ..., 243, 243, 243]], dtype=uint8)
 array([[ 37,  37,  37, ..., 199, 199, 198],
       [ 37,  37,  38, ..., 199, 199, 198],
       [ 37,  38,  38, ..., 199, 199, 198],
       ...,
       [ 15,  15,  16, ..., 138, 138, 138],
       [ 15,  15,  16, ..., 138, 138, 138],
       [ 12,  16,  18, ..., 136, 136, 137]], dtype=uint8)
 array([[156, 156, 158, ..., 161, 159, 159],
       [158, 159, 160, ..., 161, 160, 159],
       [162, 162, 163, ..., 162, 161, 161],
       ...,
       [128, 128, 128, ..., 241, 241, 241],
       [129, 129, 129, ..., 240, 240, 240],
       [132, 132, 132, ..., 239, 239, 239]], dtype=uint8)
 array([[ 59,  59,  60, ...,  46,  47,  48],
       [ 60,  60,  61, ...,  49,  50,  51],
       [ 61,  62,  62, ...,  53,  54, 

In [8]:
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T

In [9]:
label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = [label_0, label_1]

In [10]:
dev = qml.device("default.qubit", wires=1)

In [11]:
@qml.qnode(dev)
def qcircuit(params, x=None, y=None):
    """A variational quantum circuit representing the Universal classifier.

    Args:
        params (array[float]): array of parameters
        x (array[float]): single input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    for p in params:
        qml.Rot(*x, wires=0)
        qml.Rot(*p, wires=0)
    return qml.expval(qml.Hermitian(y, wires=[0]))

In [12]:
def cost(params, x, y, state_labels=None):
    """Cost function to be minimized.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d array of input vectors
        y (array[float]): 1-d array of targets
        state_labels (array[float]): array of state representations for labels

    Returns:
        float: loss value to be minimized
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        if y[i] == 1:
            y_use = dm_labels[0]
        elif y[i] == 0:
            y_use = dm_labels[1]
        for j in range(len(x[i])):
            f = qcircuit(params, x=[x[i][j][0],x[i][j][1],x[i][j][2]], y=y_use)
            loss = loss + (1 - f) ** 2
    return loss / (len(x)*len(x[i]))

In [13]:
def test(params, x, y, state_labels=None):

    #Tests on a given set of data.

    #Args:
    #    params (array[float]): array of parameters
    #    x (array[float]): 2-d array of input vectors
    #    y (array[float]): 1-d array of targets
    #    state_labels (array[float]): 1-d array of state representations for labels

    #Returns:
    #    predicted (array([int]): predicted labels for test data
    #    output_states (array[float]): output quantum states from the circuit

    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        for j in range(len(x[i])):
            fidel_function = lambda y: qcircuit(params, x=[x[i][j][0],x[i][j][1],x[i][j][2]], y=y)
            fidelities = [fidel_function(dm) for dm in dm_labels]
            best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(predicted), np.array(fidelity_values)

In [14]:
def accuracy_score(y_true, y_pred):
    """Accuracy score.

    Args:
        y_true (array[float]): 1-d array of targets
        y_predicted (array[float]): 1-d array of predictions
        state_labels (array[float]): 1-d array of state representations for labels

    Returns:
        score (float): the fraction of correctly classified samples
    """
    print(y_true)
    print(y_pred)
    score = y_true == y_pred
    return score.sum() / len(y_true)

In [15]:
def iterate_minibatches(inputs, targets, batch_size):
    """
    A generator for batches of the input data

    Args:
        inputs (array[float]): input data
        targets (array[float]): targets

    Returns:
        inputs (array[float]): one batch of input data of length `batch_size`
        targets (array[float]): one batch of targets of length `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [16]:
# Generate training and test data
num_training = 200
num_test = 2000

# Train using Adam optimizer and evaluate the classifier
num_layers = 3
learning_rate = 0.7
epochs = 10
batch_size = 2

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

# initialize random weights
params = np.random.uniform(size=(num_layers, 3))

predicted_train, fidel_train = test(params, images, labels, state_labels)
accuracy_train = accuracy_score(labels, predicted_train)

predicted_test, fidel_test = test(params, images, labels, state_labels)
accuracy_test = accuracy_score(labels, predicted_test)

# save predictions with random weights for comparison
initial_predictions = predicted_test

loss = cost(params, images, labels, state_labels)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test
    )
)

for it in range(epochs):
    for Xbatch, ybatch in iterate_minibatches(images, labels, batch_size=batch_size):
        params = opt.step(lambda v: cost(v, Xbatch, ybatch, state_labels), params)

    predicted_train, fidel_train = test(params, images, labels, state_labels)
    accuracy_train = accuracy_score(labels, predicted_train)
    loss = cost(params, images, labels, state_labels)

    predicted_test, fidel_test = test(params, images, labels, state_labels)
    accuracy_test = accuracy_score(labels, predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )
    print(predicted_test)

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0 1 0 0 1 0 1 0 1 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0 1 0 0 1 0 1 0 1 0]
Epoch:  0 | Cost: 1881.824534 | Train accuracy: 0.500000 | Test Accuracy: 0.500000
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1 1 1 0 1 1 1 0 1 1]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1 1 1 0 1 1 1 0 1 1]
Epoch:  1 | Loss: 1349.770286 | Train accuracy: 0.500000 | Test accuracy: 0.500000
[1 1 1 0 1 1 1 0 1 1]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0 1 0 0 0 1 1 0 0 1]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0 1 0 0 0 1 1 0 0 1]
Epoch:  2 | Loss: 1559.014553 | Train accuracy: 0.300000 | Test accuracy: 0.300000
[0 1 0 0 0 1 1 0 0 1]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0 1 0 0 0 1 1 1 0 1]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0 1 0 0 0 1 1 1 0 1]
Epoch:  3 | Loss: 1437.450334 | Train accuracy: 0.200000 | Test accuracy: 0.200000
[0 1 0 0 0 1 1 1 0 1]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1 1 1 0 1 1 0 1 1 1]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1 1 1 0 1 1 0 1 1 1]
Epoch:  4 | Loss: 1110.740260 | Train accuracy: 0.500000 | Test accuracy